In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
# 载入数据集
# 没有的话会自动去下载数据集
# 数据路径  转为one_hot(某一位数字为1，其余数字都为0)格式
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
# 运行次数
max_steps = 1001
# 图片数量
image_num = 3000
# 文件路径
DIR = './'
# 载入图片
'''
# 'x' is [1, 4]
# 'y' is [2, 5]
# 'z' is [3, 6]
# 默认 -> 行
stack([x, y, z]) => [[1, 4], [2, 5], [3, 6]]  # Pack along first dim.
# 列
stack([x, y, z], axis=1) => [[1, 2, 3], [4, 5, 6]]
'''
# mnist.test.images[:image_num] -> 从 0 开始到 3000
# 图片数据共有 一万 张 最大不能超多 10000
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), trainable=False, name='embedding')

# 参数概要
def variable_summaries(var):
    # 命名空间
    with tf.name_scope('summaries'):
        # 平均值
        mean = tf.reduce_mean(var)
        # summary.scalar -> 记录值，且指定相应的名称
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        # 标准差
        tf.summary.scalar('stddev', stddev)
        # 最大值
        tf.summary.scalar('max', tf.reduce_max(var))
        # 最小值
        tf.summary.scalar('min', tf.reduce_min(var))
        # 直方图
        tf.summary.histogram('histogram', var)

# 命名空间
with tf.name_scope('input'):
    # None -> 第一个维度可以是任意的长度
    x = tf.placeholder(tf.float32, [None, 784], name='x_input')
    # 真实值 -> 正确的标签
    y = tf.placeholder(tf.float32, [None, 10], name='y_input')

# 显示图片
with tf.name_scope('input_reshape'):
    # reshape
    # -1 代表不确定的值 
    # 将 784 转换为 28行 28列
    # 黑白色图片 -> 维度为 1
    # 彩色图片 -> 维度为 3
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    # 传递参数，放 10 张图片
    tf.summary.image('input', image_shaped_input, 10)

# 隐藏层
with tf.name_scope('layer'):
    # 创建一个简单神经网络
    # 权值
    with tf.name_scope('weights'):
        W = tf.Variable(tf.zeros([784, 10]), name='W')
        variable_summaries(W)
    # 偏置值
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]), name='b')
        variable_summaries(b)
    # softmax -> 交叉熵函数激活
    # matmul -> 矩阵相乘
    # matmul(数据和权值) + 偏置值 再 使用softmax函数激活
    # softmax -> 转换为概率值
    with tf.name_scope('softmax'):
        # 预测
        prediction = tf.nn.softmax(tf.matmul(x, W) + b)
        
# 最小化loss
with tf.name_scope('loss'):
    # tf.no.softmax_cross_entropy_with_logits -> 交叉熵代价函数
    # params -> 标签值(真实值), 预测值
    # tf.reduce_mean -> 平均值
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):
    # 梯度下降法优化
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
# 测试数据
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # 测试准确率 => 存放布尔值的列表
        # tf.equal -> 比较参数一(真实值数据)和参数二(预测值数据)行[或者列]的最大值 => True or False
        # argmax -> 返回一维张量中最大值所在的位置
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
    with tf.name_scope('accuracy'):
        # 求准确率
        # tf.cast -> 将对比后的布尔值列表转换为对应的浮点值 => True为1.0，False为0
        # tf.reduce_mean -> 平均值
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# 创建会话
with tf.Session() as sess:
    # 初始化变量
    sess.run(tf.global_variables_initializer())
    
    # 产生metadata文件
    if tf.gfile.Exists(DIR + 'projector/projector/metadata.tsv'):
        tf.gfile.DeleteRecursively(DIR + 'projector/projector/metadata.tsv')
    with open(DIR + 'projector/projector/metadata.tsv', 'w') as f:
        # 拿到测试集里面的标签
        # argmax -> 返回一维张量中最大值所在的位置 => 就相当于图片的标签
        labels = sess.run(tf.argmax(mnist.test.labels[:], 1))
        for i in range(image_num):
            f.write(str(labels[i]) + '\n')

    # 合并所有的summary
    merged = tf.summary.merge_all()

    '''
    画 params => 路径, 图结构(graph)
    '''
    projector_writer = tf.summary.FileWriter(DIR + 'projector/projector', sess.graph)
    # 用于保存网络模型
    saver = tf.train.Saver()
    # 配置项
    config = projector.ProjectorConfig()
    # 设置配置项
    embed = config.embeddings.add()
    embed.tensor_name = embedding.name
    embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
    embed.sprite.image_path = DIR + 'projector/data/mnist_10k_sprite.png'
    # 切割每一个图片的像素点
    embed.sprite.single_image_dim.extend([28, 28])
    # 可视化
    projector.visualize_embeddings(projector_writer, config)
    # 训练
    for i in range(max_steps):
        # 每个批次100个样本
        batch_xs, batch_ys = mnist.train.next_batch(100)
        # 配置
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
        # 执行训练传入数据
        summary, _ = sess.run([merged, train_step], feed_dict={x:batch_xs, y:batch_ys}, options=run_options, run_metadata=run_metadata)
        # 配置
        projector_writer.add_run_metadata(run_metadata, 'step%03d' % i)
        projector_writer.add_summary(summary, i)
        
        if i % 100 == 0:
            # 测试准确率
            acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
            print('Iter %s \t Testing Accuracy %s' % (i, acc))
    
    saver.save(sess, DIR + 'projector/projector/a_model.ckpt', global_step=max_steps)
    projector_writer.close()

Iter 0 	 Testing Accuracy 0.316
Iter 100 	 Testing Accuracy 0.8063
Iter 200 	 Testing Accuracy 0.8213
Iter 300 	 Testing Accuracy 0.8582
Iter 400 	 Testing Accuracy 0.8873
Iter 500 	 Testing Accuracy 0.8963
Iter 600 	 Testing Accuracy 0.8986
Iter 700 	 Testing Accuracy 0.9016
Iter 800 	 Testing Accuracy 0.9042
Iter 900 	 Testing Accuracy 0.9046
Iter 1000 	 Testing Accuracy 0.9079
